In [4]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"

pickle_path = "../../data/tencent2020/pickle"
if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
pd.set_option('display.max_columns', None)

In [5]:
train_click_log = pd.read_csv(path_save + "train_click_log.csv", encoding="utf-8")
test_click_log = pd.read_csv(path_save + "test_click_log.csv", encoding="utf-8")
train_click_log.shape, test_click_log.shape

((30082771, 9), (33585512, 9))

train：user_id 范围 (1, 900000)  
test ：user_id 范围 (3000001, 4000000) 

In [8]:
click_log = train_click_log.append(test_click_log)
click_log.sort_values(by=['user_id','time'], ascending=[True,True], inplace=True)
click_log = click_log.reset_index(drop=True)
click_log = click_log[click_log['click_times']<=8] #去除点击次数99.99%外的数据记录
print(click_log.shape)
click_log.head()

(63663353, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,1,821396,1,724607,0,5,7293,326
1,20,1,209778,1,188507,136,2,9702,6
2,20,1,877468,1,773445,0,5,29455,106
3,39,1,1683713,1,1458878,0,5,14668,326
4,40,1,122032,1,109959,1334,2,11411,0


In [ ]:
time_df = click_log.groupby([''])

In [9]:
#click_log['time_7'] = click_log['time']//7
# click_log['creative_id_times'] = click_log['creative_id']*10000000 + click_log['click_times']
# click_log['ad_id_times'] = click_log['ad_id']*10000000 + click_log['click_times']
# click_log['product_id_times'] = click_log['product_id']*10000000 + click_log['click_times']
# click_log['advertiser_id_times'] = click_log['advertiser_id']*10000000 + click_log['click_times']
# click_log['product_category_times'] = click_log['product_category']*10000000 + click_log['click_times']
# click_log['industry_times'] = click_log['industry']*10000000 + click_log['click_times']


click_log['creative_id_times'] = click_log['creative_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['ad_id_times'] = click_log['ad_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['product_id_times'] = click_log['product_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['advertiser_id_times'] = click_log['advertiser_id'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['product_category_times'] = click_log['product_category'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')
click_log['industry_times'] = click_log['industry'].astype(str).str.cat(click_log['click_times'].astype(str), sep='-')


In [10]:
click_log = multi_column_LabelEncoder(click_log, 
                                      columns=['creative_id','ad_id', 'product_id', 'product_category', 'advertiser_id','industry' ,
                                               'creative_id_times','ad_id_times','product_id_times',
                                               'advertiser_id_times','product_category_times', 'industry_times',
                                               
                                              ])
# click_log = multi_column_LabelEncoderOrder(click_log, 
#                                       columns=['creative_id_times','ad_id_times','product_id_times',
#                                                'advertiser_id_times','product_category_times', 'industry_times',   
#                                                 'creative_id_t','ad_id_t','product_id_t',
#                                                 'advertiser_id_t','product_category_t','industry_t',                                             
#                                               ], max_features=2000000)

print(click_log['creative_id'].nunique())
print(click_log['ad_id'].nunique())
print(click_log['product_id'].nunique())
print(click_log['product_category'].nunique())
print(click_log['advertiser_id'].nunique())
print(click_log['industry'].nunique())
print(click_log['creative_id_times'].nunique())
print(click_log['ad_id_times'].nunique())
print(click_log['product_id_times'].nunique())
print(click_log['advertiser_id_times'].nunique())
print(click_log['product_category_times'].nunique())
print(click_log['industry_times'].nunique())

# print(click_log['creative_id_t'].nunique())
# print(click_log['ad_id_t'].nunique())
# print(click_log['product_id_t'].nunique())
# print(click_log['advertiser_id_t'].nunique())
# print(click_log['product_category_t'].nunique())
# print(click_log['industry_t'].nunique())

creative_id LabelEncoder......
ad_id LabelEncoder......
product_id LabelEncoder......
product_category LabelEncoder......
advertiser_id LabelEncoder......
industry LabelEncoder......
creative_id_times LabelEncoder......
ad_id_times LabelEncoder......
product_id_times LabelEncoder......
advertiser_id_times LabelEncoder......
product_category_times LabelEncoder......
industry_times LabelEncoder......
LabelEncoder Successfully!
3412767
3027358
39057
18
57870
332
4061140
3659810
57398
104101
113
1690


In [11]:
user_ids = click_log.groupby(['user_id']).agg({'creative_id':lambda x:list(x),
                                                        'ad_id':lambda x:list(x),
                                                        'product_id':lambda x:list(x),
                                                        'product_category':lambda x:list(x),
                                                        'advertiser_id':lambda x:list(x),
                                                        'industry':lambda x:list(x),   
                                                        'click_times':lambda x:list(x),
                                                         'time':lambda x:list(x),
                                                'creative_id_times':lambda x:list(x),
                                                'ad_id_times':lambda x:list(x),
                                                'product_id_times':lambda x:list(x),
                                                'advertiser_id_times':lambda x:list(x),
                                                'product_category_times':lambda x:list(x),
                                                'industry_times':lambda x:list(x),
#                                                'creative_id_t':lambda x:list(x),
#                                                 'ad_id_t':lambda x:list(x),
#                                                 'product_id_t':lambda x:list(x),
#                                                 'advertiser_id_t':lambda x:list(x),
#                                                 'product_category_t':lambda x:list(x),
#                                                 'industry_t':lambda x:list(x),
                                                       }).reset_index()

In [12]:
user_ids['creative_id_len'] = user_ids['creative_id'].map(lambda x: len(set(x)))
user_ids['ad_id_len'] = user_ids['ad_id'].map(lambda x: len(set(x)))
user_ids['product_id_len'] = user_ids['product_id'].map(lambda x: len(set(x)))
user_ids['product_category_len'] = user_ids['product_category'].map(lambda x: len(set(x)))
user_ids['advertiser_len'] = user_ids['advertiser_id'].map(lambda x: len(set(x)))
user_ids['industry_len'] = user_ids['industry'].map(lambda x: len(set(x)))
user_ids['mean_clicktimes'] = user_ids['click_times'].map(lambda x: np.mean(x))
user_ids['max_clicktimes'] = user_ids['click_times'].map(lambda x: np.max(x))
user_ids['min_clicktimes'] = user_ids['click_times'].map(lambda x: np.min(x))

user_ids['mean_time'] = user_ids['time'].map(lambda x: np.mean(x))
user_ids['max_time'] = user_ids['time'].map(lambda x: np.max(x))
user_ids['min_time'] = user_ids['time'].map(lambda x: np.min(x))

user_ids['time_len'] = user_ids['time'].map(lambda x: len(set(x)))

user_ids.to_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user_ids.head(2)


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,ad_id_times,product_id_times,advertiser_id_times,product_category_times,industry_times,creative_id_len,ad_id_len,product_id_len,product_category_len,advertiser_len,industry_len,mean_clicktimes,max_clicktimes,min_clicktimes,mean_time,max_time,min_time,time_len
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[3872964, 1123872, 3935014, 706993, 226871, 37...","[3358402, 950010, 3415179, 499291, 107496, 328...","[1, 6399, 1, 1, 5954, 1, 1, 1, 1, 8147, 52204,...","[99263, 103509, 38769, 8445, 2750, 8484, 13490...","[82, 59, 82, 82, 59, 51, 82, 82, 51, 59, 59, 5...","[1309, 1509, 41, 1309, 1, 1309, 1570, 634, 154...",12,12,6,3,12,9,1.076923,2,1,47.461538,76,20,10
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[3670932, 575824, 3013599, 3644043, 316325, 34...","[3202365, 430643, 2970933, 3151275, 488750, 29...","[54699, 53814, 5157, 5157, 7149, 54699, 1, 462...","[25317, 1326, 16287, 31864, 99902, 25317, 4850...","[59, 59, 59, 59, 59, 59, 51, 59, 51, 51, 59, 5...","[1252, 780, 1509, 1509, 1244, 1252, 1442, 1509...",42,42,20,3,36,15,1.022222,2,1,53.822222,90,10,28
